In [1]:
from pyspark.sql import SparkSession
from cassandra.cluster import Cluster, PlainTextAuthProvider
from cassandra.query import SimpleStatement

In [2]:
packages = {
    "com.datastax.spark:spark-cassandra-connector_2.12:3.3.0"
}

In [3]:
spark = (
    SparkSession.builder
    .appName("cassandraTest")
    .config("spark.jars.packages", ",".join(packages))
    .config("spark.cassandra.connection.host", "cassandra")
    .config("spark.cassandra.connection.port", "9042")
    .config("spark.cassandra.auth.username", "cassandra")
    .config("spark.cassandra.auth.password", "cassandra")
    .config("spark.sql.catalog.myCatalog", "com.datastax.spark.connector.datasource.CassandraCatalog")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c76ebad-fbc6-4c45-9f76-ee88eaaa9a1c;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.3.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.3.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	

23/04/06 13:38:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(['cassandra'], port=9042, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.execute("DROP TABLE IF EXISTS mykeyspace.mytable")
session.execute("CREATE TABLE IF NOT EXISTS mykeyspace.mytable (id int PRIMARY KEY, name text, age int)")


In [6]:
insert_statement = session.prepare("INSERT INTO mykeyspace.mytable (id, name, age) VALUES (?, ?, ?);")
for i in range(5):
    session.execute(insert_statement, (i, f"Name{i}", i*10))

In [10]:
df = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="mytable", keyspace="mykeyspace")
    .load()
)
df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [11]:
df.show()

+---+---+-----+
| id|age| name|
+---+---+-----+
|  0|  0|Name0|
|  2| 20|Name2|
|  1| 10|Name1|
|  3| 30|Name3|
|  4| 40|Name4|
+---+---+-----+

